# Exercise Guide #3: Speaking HTTP

## Exercise: Manual Web Client

Let's talk to Google! In a new terminal, type,

```
nc google.com 80
GET /
```
and hit enter twice at the end.

You should get back something interesting!

To review, let's try that in Python. In a Python prompt, type exactly these lines:

```
>>> import socket
>>> s = socket.socket()
>>> s.connect(('google.com', 80))
>>> s.send('GET /\n\n')
>>> s.recv(1000)
>>> s.recv(1000)
```

We're speaking HTTP! A more complete example of an HTTP request might look like this:
```
GET /index.html HTTP/1.1
Host: www.example.com
```
but you don't need to type that.

## Exercise: Manual Web Server

In a terminal, type:
```
nc -l 8000
```
Now in a web browser, type `localhost:8000/hello` into the location bar.

Back in your terminal, type exactly these lines: (note the single blank line (meaning pressing return twice) after the first line.
```
HTTP/1.1 200 OK

<html>
<h1>Hello!</h1>
</html>
```
and then press control-d to close the connection. On Windows it might be control-c? Or control-z?

## Project: Write a web server
Here's the outline of a simple, noncompliant http server.
```
import socket

listener = socket.socket()
listener.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
listener.bind((localhost, 8000))
listener.listen(5)

while True:
    s, addr = listener.accept()
    request = s.recv(1000)

    #parsing the http request
    method, rest = request.split(b' ', 1)
    path, rest = rest.split(None, 1)

    s.send(b'HTTP/1.0 200 OK\n')
    # send any other headers here 
    s.send(b'\n')

    # change this line to send a file here
    s.send(b'<html><h1>hey</h1</html>')

    s.close()
```

We want to turn this into a file server: a program that responds to http requests with the file requested.

* open the file in binary mode: `open(filename, 'rb')`
* since it's unlikely the whole file will fit in a single send call, either use socket.sendall() or write a loop yourself to check the return value of socket.send()

See the example HTTP session on WIkipedia for what a request and a response might look like, or refer to the request you saw with `nc -l` and the response you saw with `nc google.com 80`.

Extensions beyond returning the bytes of the file:

* Include the 'Content-Type:' header in the response with the mime type (see the mimetypes module, https://docs.python.org/3.6/library/mimetypes.html)
* Include the 'Content-Length:' header
* Respond with "404 Not Found" instead of "200 OK" if the file is not found
* Respond with an html 404 page
* Respond with an index page (a list of files) if the requested path is a directory

Here's the original http "spec" before there was a proper RFC for it: https://www.w3.org/Protocols/HTTP/AsImplemented.html